In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

import sys
sys.path.append('..')

from src.beam import beam_arguments, Experiment
from src.beam import UniversalDataset, UniversalBatchSampler
from src.beam import Algorithm
from src.beam import LinearNet

from src.beam import DataTensor
from src.beam.utils import is_notebook

from mnist_example import run_mnist, mnist_algorithm_generator

In [2]:
# path_to_data = '/localdata/elads/data/datasets/mnist'
# root_dir = '/localdata/elads/data/resutls'

path_to_data = '/home/shared/data//dataset/mnist'
root_dir = '/home/shared/data/results'

## Training with a single worker

In [3]:
# here you put all actions which are performed only once before initializing the workers
# for example, setting running arguments and experiment:

args = beam_arguments(f"--project-name=mnist --root-dir={root_dir} --algorithm=MNISTAlgorithm",
                      "--epoch-length=100000 --n-epochs=2 --clip=1 --parallel=1", path_to_data=path_to_data)

experiment = Experiment(args)

2022-06-07 14:18:55.974 | INFO     | src.beam.experiment:__init__:94 - beam project: mnist
2022-06-07 14:18:55.976 | INFO     | src.beam.experiment:__init__:95 - Simulation Hyperparameters
2022-06-07 14:18:55.977 | INFO     | src.beam.experiment:__init__:99 - project_name: mnist
2022-06-07 14:18:55.978 | INFO     | src.beam.experiment:__init__:99 - identifier: debug
2022-06-07 14:18:55.979 | INFO     | src.beam.experiment:__init__:99 - algorithm: MNISTAlgorithm
2022-06-07 14:18:55.980 | INFO     | src.beam.experiment:__init__:99 - mp_port: None
2022-06-07 14:18:55.981 | INFO     | src.beam.experiment:__init__:99 - root_dir: /home/shared/data/results
2022-06-07 14:18:55.982 | INFO     | src.beam.experiment:__init__:99 - reload: False
2022-06-07 14:18:55.983 | INFO     | src.beam.experiment:__init__:99 - resume: -1
2022-06-07 14:18:55.984 | INFO     | src.beam.experiment:__init__:99 - override: False
2022-06-07 14:18:55.985 | INFO     | src.beam.experiment:__init__:99 - cpu_workers: 0
20

we can generate an untrained algorithm with

In [4]:
alg = mnist_algorithm_generator(experiment)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /home/shared/data//dataset/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /home/shared/data//dataset/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /home/shared/data//dataset/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /home/shared/data//dataset/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /home/shared/data//dataset/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/shared/data//dataset/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /home/shared/data//dataset/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/shared/data//dataset/mnist/MNIST/raw



In [5]:
alg.fit()

2022-06-07 13:33:34.767 | INFO     | src.beam.experiment:run:425 - Single worker mode
2022-06-07 13:33:34.768 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/1 is running...


2022-06-07 13:33:34.904650: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


train:   0%|          | 0/389 [00:00<?, ?it/s]

validation:   0%|          | 0/389 [00:00<?, ?it/s]


2022-06-07 13:33:37.734 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 1/2:
2022-06-07 13:33:37.739 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-07 13:33:37.739 | INFO     | src.beam.experiment:log_data:344 - loss 0.336389 	|
2022-06-07 13:33:37.740 | INFO     | src.beam.experiment:log_data:344 - acc 0.904924 	|
2022-06-07 13:33:37.740 | INFO     | src.beam.experiment:log_data:344 - lr 0.00099 	|
2022-06-07 13:33:37.741 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-07 13:33:37.741 | INFO     | src.beam.experiment:log_data:344 - loss 0.148709 	|
2022-06-07 13:33:37.742 | INFO     | src.beam.experiment:log_data:344 - acc 0.955816 	|


train:   0%|          | 0/389 [00:00<?, ?it/s]

validation:   0%|          | 0/389 [00:00<?, ?it/s]


2022-06-07 13:33:39.606 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 2/2:
2022-06-07 13:33:39.611 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-07 13:33:39.611 | INFO     | src.beam.experiment:log_data:344 - loss 0.0909585 	|
2022-06-07 13:33:39.612 | INFO     | src.beam.experiment:log_data:344 - acc 0.972395 	|
2022-06-07 13:33:39.612 | INFO     | src.beam.experiment:log_data:344 - lr 0.0009801 	|
2022-06-07 13:33:39.613 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-07 13:33:39.613 | INFO     | src.beam.experiment:log_data:344 - loss 0.104674 	|
2022-06-07 13:33:39.614 | INFO     | src.beam.experiment:log_data:344 - acc 0.967886 	|


or we can apply the default training routin with

In [6]:
alg = experiment(mnist_algorithm_generator)

2022-06-07 13:33:44.446 | INFO     | src.beam.experiment:run:425 - Single worker mode
2022-06-07 13:33:44.448 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/1 is running...


train:   0%|          | 0/389 [00:00<?, ?it/s]

validation:   0%|          | 0/389 [00:00<?, ?it/s]


2022-06-07 13:33:46.211 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 3/2:
2022-06-07 13:33:46.216 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-07 13:33:46.217 | INFO     | src.beam.experiment:log_data:344 - loss 0.340606 	|
2022-06-07 13:33:46.217 | INFO     | src.beam.experiment:log_data:344 - acc 0.90385 	|
2022-06-07 13:33:46.218 | INFO     | src.beam.experiment:log_data:344 - lr 0.00099 	|
2022-06-07 13:33:46.218 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-07 13:33:46.219 | INFO     | src.beam.experiment:log_data:344 - loss 0.136995 	|
2022-06-07 13:33:46.219 | INFO     | src.beam.experiment:log_data:344 - acc 0.958568 	|


train:   0%|          | 0/389 [00:00<?, ?it/s]

validation:   0%|          | 0/389 [00:00<?, ?it/s]


2022-06-07 13:33:47.820 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 4/2:
2022-06-07 13:33:47.823 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-07 13:33:47.824 | INFO     | src.beam.experiment:log_data:344 - loss 0.0876803 	|
2022-06-07 13:33:47.824 | INFO     | src.beam.experiment:log_data:344 - acc 0.973429 	|
2022-06-07 13:33:47.825 | INFO     | src.beam.experiment:log_data:344 - lr 0.0009801 	|
2022-06-07 13:33:47.825 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-07 13:33:47.825 | INFO     | src.beam.experiment:log_data:344 - loss 0.0971943 	|
2022-06-07 13:33:47.826 | INFO     | src.beam.experiment:log_data:344 - acc 0.970849 	|


we can now access the trained algorithm

In [8]:
alg.networks['net']

LinearNet(
  (lin): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=10, bias=True)
  )
)

## Inference

In [9]:
inference = alg('test')

print('Test inference results:')
for n, v in inference['metrics'].items():
    print(f'{n}:')
    print(v)

test:   0%|          | 0/39 [00:00<?, ?it/s]

Test inference results:
precision:
[0.97585513 0.99115044 0.97010608 0.95472062 0.97357724 0.96524664
 0.97782471 0.94647887 0.97671958 0.9803719 ]
recall:
[0.98979592 0.98678414 0.9748062  0.98118812 0.97556008 0.96524664
 0.96659708 0.98054475 0.9476386  0.94053518]
fscore:
[0.98277609 0.98896247 0.97245046 0.96777344 0.97456765 0.96524664
 0.97217848 0.9632107  0.96195935 0.96004047]
support:
[ 980 1135 1032 1010  982  892  958 1028  974 1009]


we can also run our own experiment routine and obtain the results

In [10]:
alg, results = experiment.run(run_mnist)

2022-06-01 19:58:59.572 | INFO     | src.beam.experiment:run:425 - Single worker mode
2022-06-01 19:58:59.573 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/1 is running...


train:   0%|          | 0/389 [00:00<?, ?it/s]

validation:   0%|          | 0/389 [00:00<?, ?it/s]


2022-06-01 19:59:01.451 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 3/2:
2022-06-01 19:59:01.456 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-01 19:59:01.457 | INFO     | src.beam.experiment:log_data:344 - loss 0.333092 	|
2022-06-01 19:59:01.458 | INFO     | src.beam.experiment:log_data:344 - acc 0.904935 	|
2022-06-01 19:59:01.459 | INFO     | src.beam.experiment:log_data:344 - lr 0.00099 	|
2022-06-01 19:59:01.460 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-01 19:59:01.461 | INFO     | src.beam.experiment:log_data:344 - loss 0.136295 	|
2022-06-01 19:59:01.462 | INFO     | src.beam.experiment:log_data:344 - acc 0.958618 	|


train:   0%|          | 0/389 [00:00<?, ?it/s]

validation:   0%|          | 0/389 [00:00<?, ?it/s]


2022-06-01 19:59:03.455 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 4/2:
2022-06-01 19:59:03.459 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-01 19:59:03.460 | INFO     | src.beam.experiment:log_data:344 - loss 0.0878459 	|
2022-06-01 19:59:03.461 | INFO     | src.beam.experiment:log_data:344 - acc 0.973289 	|
2022-06-01 19:59:03.461 | INFO     | src.beam.experiment:log_data:344 - lr 0.0009801 	|
2022-06-01 19:59:03.462 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-01 19:59:03.463 | INFO     | src.beam.experiment:log_data:344 - loss 0.0949364 	|
2022-06-01 19:59:03.463 | INFO     | src.beam.experiment:log_data:344 - acc 0.971692 	|


In [11]:
np.mean(results['validation']['scalar']['loss'])

0.09493644842949808

## Inference

In [12]:
inference = alg('test')

print('Test inference results:')
for n, v in inference['metrics'].items():
    print(f'{n}:')
    print(v)

test:   0%|          | 0/39 [00:00<?, ?it/s]

Test inference results:
precision:
[0.97683787 0.98423818 0.97019231 0.94676806 0.960199   0.98484848
 0.96707819 0.9707887  0.9697286  0.98740818]
recall:
[0.98979592 0.99030837 0.97771318 0.98613861 0.98268839 0.94730942
 0.98121086 0.96984436 0.95379877 0.93260654]
fscore:
[0.9832742  0.98726394 0.97393822 0.96605238 0.97131354 0.96571429
 0.97409326 0.9703163  0.96169772 0.95922528]
support:
[ 980 1135 1032 1010  982  892  958 1028  974 1009]


## Reload algorithm and resume training

In [16]:
# here you put all actions which are performed only once before initializing the workers
# for example, setting running arguments and experiment:

args = beam_arguments(f"--project-name=mnist --root-dir={root_dir} --algorithm=MNISTAlgorithm",
                      "--epoch-length=100000 --n-epochs=2 --clip=1 --parallel=1 --reload", 
                      path_to_data=path_to_data)

experiment = Experiment(args)

2022-06-01 20:00:10.595 | INFO     | src.beam.experiment:__init__:94 - beam project: mnist
2022-06-01 20:00:10.596 | INFO     | src.beam.experiment:__init__:95 - Simulation Hyperparameters
2022-06-01 20:00:10.598 | INFO     | src.beam.experiment:__init__:99 - project_name: mnist
2022-06-01 20:00:10.599 | INFO     | src.beam.experiment:__init__:99 - identifier: debug
2022-06-01 20:00:10.600 | INFO     | src.beam.experiment:__init__:99 - algorithm: MNISTAlgorithm
2022-06-01 20:00:10.601 | INFO     | src.beam.experiment:__init__:99 - mp_port: None
2022-06-01 20:00:10.602 | INFO     | src.beam.experiment:__init__:99 - root_dir: /localdata/elads/data/resutls
2022-06-01 20:00:10.603 | INFO     | src.beam.experiment:__init__:99 - reload: True
2022-06-01 20:00:10.605 | INFO     | src.beam.experiment:__init__:99 - resume: -1
2022-06-01 20:00:10.606 | INFO     | src.beam.experiment:__init__:99 - override: False
2022-06-01 20:00:10.607 | INFO     | src.beam.experiment:__init__:99 - cpu_workers: 0

In [17]:
alg = experiment(mnist_algorithm_generator)

2022-06-01 20:00:13.889 | INFO     | src.beam.experiment:run:425 - Single worker mode
2022-06-01 20:00:13.891 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/1 is running...
2022-06-01 20:00:13.989 | INFO     | src.beam.experiment:reload_checkpoint:245 - Reload experiment from checkpoint: /localdata/elads/data/resutls/mnist/MNISTAlgorithm/debug/0001_20220601_194628/checkpoints/checkpoint_000004


train:   0%|          | 0/389 [00:00<?, ?it/s]

validation:   0%|          | 0/389 [00:00<?, ?it/s]


2022-06-01 20:00:15.814 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 1/2:
2022-06-01 20:00:15.820 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-01 20:00:15.820 | INFO     | src.beam.experiment:log_data:344 - loss 0.0471516 	|
2022-06-01 20:00:15.821 | INFO     | src.beam.experiment:log_data:344 - acc 0.985791 	|
2022-06-01 20:00:15.821 | INFO     | src.beam.experiment:log_data:344 - lr 0.000970299 	|
2022-06-01 20:00:15.822 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-01 20:00:15.823 | INFO     | src.beam.experiment:log_data:344 - loss 0.092408 	|
2022-06-01 20:00:15.823 | INFO     | src.beam.experiment:log_data:344 - acc 0.973329 	|


train:   0%|          | 0/389 [00:00<?, ?it/s]

validation:   0%|          | 0/389 [00:00<?, ?it/s]


2022-06-01 20:00:17.862 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 2/2:
2022-06-01 20:00:17.867 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-01 20:00:17.867 | INFO     | src.beam.experiment:log_data:344 - loss 0.0272187 	|
2022-06-01 20:00:17.868 | INFO     | src.beam.experiment:log_data:344 - acc 0.991716 	|
2022-06-01 20:00:17.869 | INFO     | src.beam.experiment:log_data:344 - lr 0.000960596 	|
2022-06-01 20:00:17.869 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-01 20:00:17.870 | INFO     | src.beam.experiment:log_data:344 - loss 0.100829 	|
2022-06-01 20:00:17.871 | INFO     | src.beam.experiment:log_data:344 - acc 0.972496 	|


## Training with two workers

In [19]:
# here you put all actions which are performed only once before initializing the workers
# for example, setting running arguments and experiment:

args = beam_arguments(f"--project-name=mnist --root-dir={root_dir} --algorithm=MNISTAlgorithm",
                      "--epoch-length=100000 --n-epochs=2 --clip=1 --parallel=2",
                      path_to_data=path_to_data)

experiment = Experiment(args)

2022-06-01 20:00:57.478 | INFO     | src.beam.experiment:__init__:94 - beam project: mnist
2022-06-01 20:00:57.480 | INFO     | src.beam.experiment:__init__:95 - Simulation Hyperparameters
2022-06-01 20:00:57.481 | INFO     | src.beam.experiment:__init__:99 - project_name: mnist
2022-06-01 20:00:57.483 | INFO     | src.beam.experiment:__init__:99 - identifier: debug
2022-06-01 20:00:57.484 | INFO     | src.beam.experiment:__init__:99 - algorithm: MNISTAlgorithm
2022-06-01 20:00:57.485 | INFO     | src.beam.experiment:__init__:99 - mp_port: None
2022-06-01 20:00:57.486 | INFO     | src.beam.experiment:__init__:99 - root_dir: /localdata/elads/data/resutls
2022-06-01 20:00:57.488 | INFO     | src.beam.experiment:__init__:99 - reload: False
2022-06-01 20:00:57.489 | INFO     | src.beam.experiment:__init__:99 - resume: -1
2022-06-01 20:00:57.490 | INFO     | src.beam.experiment:__init__:99 - override: False
2022-06-01 20:00:57.491 | INFO     | src.beam.experiment:__init__:99 - cpu_workers: 

In [20]:
alg = experiment(mnist_algorithm_generator)

2022-06-01 20:01:07.034 | INFO     | src.beam.experiment:run:416 - Initializing 2 parallel workers
2022-06-01 20:01:07.036 | INFO     | src.beam.experiment:run:421 - Multiprocessing port is: 41366
2022-06-01 20:01:08.867 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/2 is running...
2022-06-01 20:01:08.882 | INFO     | src.beam.experiment:run_worker:45 - Worker: 2/2 is running...


validation: 100%|██████████| 389/389 [00:00<00:00, 1031.01it/s]


2022-06-01 20:01:17.002 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 1/2:
2022-06-01 20:01:17.008 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-01 20:01:17.009 | INFO     | src.beam.experiment:log_data:344 - loss 0.303818 	|
2022-06-01 20:01:17.009 | INFO     | src.beam.experiment:log_data:344 - acc 0.915107 	|
2022-06-01 20:01:17.009 | INFO     | src.beam.experiment:log_data:344 - lr 0.00099 	|
2022-06-01 20:01:17.009 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-01 20:01:17.009 | INFO     | src.beam.experiment:log_data:344 - loss 0.124015 	|
2022-06-01 20:01:17.009 | INFO     | src.beam.experiment:log_data:344 - acc 0.961932 	|


validation: 100%|██████████| 389/389 [00:00<00:00, 1041.04it/s]


2022-06-01 20:01:20.417 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 2/2:
2022-06-01 20:01:20.421 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-01 20:01:20.421 | INFO     | src.beam.experiment:log_data:344 - loss 0.0608674 	|
2022-06-01 20:01:20.421 | INFO     | src.beam.experiment:log_data:344 - acc 0.981674 	|
2022-06-01 20:01:20.422 | INFO     | src.beam.experiment:log_data:344 - lr 0.0009801 	|
2022-06-01 20:01:20.422 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-01 20:01:20.422 | INFO     | src.beam.experiment:log_data:344 - loss 0.0945673 	|
2022-06-01 20:01:20.422 | INFO     | src.beam.experiment:log_data:344 - acc 0.973088 	|
2022-06-01 20:01:20.840 | INFO     | src.beam.experiment:reload_checkpoint:245 - Reload experiment from checkpoint: /localdata/elads/data/resutls/mnist/MNISTAlgorithm/debug/0003_20220601_200057/checkpoints/checkpoint_000002


Inference works the same if we use the default runner, the runner reloads the stored algorithm into a single GPU mode

In [21]:
inference = alg('test')

print('Test inference results:')
for n, v in inference['metrics'].items():
    print(f'{n}:')
    print(v)

test:   0%|          | 0/39 [00:00<?, ?it/s]

Test inference results:
precision:
[0.97197197 0.98854626 0.98039216 0.98301698 0.97626419 0.99649123
 0.97401247 0.97189922 0.95070423 0.95546951]
recall:
[0.99081633 0.98854626 0.96899225 0.97425743 0.96334012 0.95515695
 0.97807933 0.97568093 0.97022587 0.97819623]
fscore:
[0.98130369 0.98854626 0.97465887 0.9786176  0.9697591  0.97538638
 0.97604167 0.97378641 0.96036585 0.96669931]
support:
[ 980 1135 1032 1010  982  892  958 1028  974 1009]


we can also determine our own experiment routine and return the results

In [22]:
# here we initialize the workers (can be single or multiple workers, depending on the configuration)
results = experiment.run(run_mnist)

2022-06-01 20:01:40.633 | INFO     | src.beam.experiment:run:416 - Initializing 2 parallel workers
2022-06-01 20:01:40.635 | INFO     | src.beam.experiment:run:421 - Multiprocessing port is: 58869
2022-06-01 20:01:42.335 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/2 is running...
2022-06-01 20:01:42.338 | INFO     | src.beam.experiment:run_worker:45 - Worker: 2/2 is running...


validation: 100%|██████████| 389/389 [00:00<00:00, 868.94it/s]


2022-06-01 20:01:50.405 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 1/2:
2022-06-01 20:01:50.441 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-01 20:01:50.441 | INFO     | src.beam.experiment:log_data:344 - loss 0.303975 	|
2022-06-01 20:01:50.441 | INFO     | src.beam.experiment:log_data:344 - acc 0.912275 	|
2022-06-01 20:01:50.442 | INFO     | src.beam.experiment:log_data:344 - lr 0.00099 	|
2022-06-01 20:01:50.442 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-01 20:01:50.442 | INFO     | src.beam.experiment:log_data:344 - loss 0.125878 	|
2022-06-01 20:01:50.443 | INFO     | src.beam.experiment:log_data:344 - acc 0.962655 	|



validation:  93%|█████████▎| 362/389 [00:00<00:00, 905.36it/s]]


2022-06-01 20:01:53.253 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 2/2:
2022-06-01 20:01:53.286 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-01 20:01:53.286 | INFO     | src.beam.experiment:log_data:344 - loss 0.0635221 	|
2022-06-01 20:01:53.286 | INFO     | src.beam.experiment:log_data:344 - acc 0.981091 	|
2022-06-01 20:01:53.286 | INFO     | src.beam.experiment:log_data:344 - lr 0.0009801 	|
2022-06-01 20:01:53.286 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-01 20:01:53.286 | INFO     | src.beam.experiment:log_data:344 - loss 0.0866934 	|
2022-06-01 20:01:53.286 | INFO     | src.beam.experiment:log_data:344 - acc 0.973359 	|


validation: 100%|██████████| 389/389 [00:00<00:00, 494.49it/s]

In [23]:
np.mean(results[1]['results']['validation']['scalar']['loss'])

0.08673303388537969

## Explainability with Lime

In [7]:
alg = mnist_algorithm_generator(experiment).fit()

2022-06-07 14:54:55.273 | INFO     | src.beam.experiment:run:425 - Single worker mode
2022-06-07 14:54:55.274 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/1 is running...


train:   0%|          | 0/389 [00:00<?, ?it/s]

validation:   0%|          | 0/389 [00:00<?, ?it/s]


2022-06-07 14:54:56.714 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 3/2:
2022-06-07 14:54:56.718 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-07 14:54:56.719 | INFO     | src.beam.experiment:log_data:344 - loss 0.341501 	|
2022-06-07 14:54:56.719 | INFO     | src.beam.experiment:log_data:344 - acc 0.901249 	|
2022-06-07 14:54:56.719 | INFO     | src.beam.experiment:log_data:344 - lr 0.00099 	|
2022-06-07 14:54:56.720 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-07 14:54:56.721 | INFO     | src.beam.experiment:log_data:344 - loss 0.142505 	|
2022-06-07 14:54:56.721 | INFO     | src.beam.experiment:log_data:344 - acc 0.956057 	|


train:   0%|          | 0/389 [00:00<?, ?it/s]

validation:   0%|          | 0/389 [00:00<?, ?it/s]


2022-06-07 14:54:58.277 | INFO     | src.beam.experiment:save_model_results:302 - Finished epoch 4/2:
2022-06-07 14:54:58.282 | INFO     | src.beam.experiment:log_data:340 - train:
2022-06-07 14:54:58.282 | INFO     | src.beam.experiment:log_data:344 - loss 0.0844115 	|
2022-06-07 14:54:58.283 | INFO     | src.beam.experiment:log_data:344 - acc 0.974373 	|
2022-06-07 14:54:58.283 | INFO     | src.beam.experiment:log_data:344 - lr 0.0009801 	|
2022-06-07 14:54:58.283 | INFO     | src.beam.experiment:log_data:340 - validation:
2022-06-07 14:54:58.284 | INFO     | src.beam.experiment:log_data:344 - loss 0.0982294 	|
2022-06-07 14:54:58.284 | INFO     | src.beam.experiment:log_data:344 - acc 0.969804 	|


In [19]:
from src.beam import UniversalDataset

In [25]:
ds = UniversalDataset(torch.randn(16, 28, 28))

In [32]:
dl = alg.build_dataloader(torch.randn(16, 28, 28))

In [31]:
type(dl)

torch.utils.data.dataloader.DataLoader

In [ ]:
next(iter(dl))

In [ ]:
results = alg.predict(torch.randn(16, 28, 28))

dataloader:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
alg.dataloaders['train'].dataset.data.shape

torch.Size([70000, 28, 28])

In [24]:
results = alg.predict('test')

test:   0%|          | 0/40 [00:00<?, ?it/s]